# Install Dependencies

## External Dependencies

In [8]:
!pip install easy-equities-client

## FrostAura Dependencies

In [9]:
import os
import sys

USE_LOCAL_CODE: bool = True # NOTE: When changing this configuration, be sure to restart the interpreter in order to have sys path updates reflect.
directories_to_move_up: int = 2
current_executing_path: str = os.getcwd()
current_executing_path = current_executing_path.replace('\\', '/')
root_path: str = '/'.join(current_executing_path.split('/')[:-directories_to_move_up])

# Clean up.
%pip uninstall frostaura -y
sys.path = [p for p in sys.path if p != root_path]

if USE_LOCAL_CODE:
    sys.path.append(root_path)
else:
    %pip install -U --no-cache-dir frostaura

Note: you may need to restart the kernel to use updated packages.


# Import Dependencies

In [10]:
from frostaura.data_access import IResourcesDataAccess
from frostaura.data_access import EmbeddedResourcesDataAccess
from frostaura.data_access import IPersonalAssetDataAccess
from frostaura.data_access import EasyEquitiesPersonalAssetDataAccess
from easy_equities_client.accounts.types import Account
import pandas as pd

# Usage

In [11]:
resource_data_access: IResourcesDataAccess = EmbeddedResourcesDataAccess()
instance: IPersonalAssetDataAccess = EasyEquitiesPersonalAssetDataAccess(resource_data_access=resource_data_access,
                                                                         username='...',
                                                                         password='...')

## Get Supported Symbols

In [12]:
supported_symbols: pd.DataFrame = instance.get_supported_assets()

supported_symbols 

,name,symbol
0,10X Genomics Inc,TXG
1,21Vianet Group Inc,VNET
2,2U Inc,TWOU
3,3D Systems Inc,DDD
4,3M Co,MMM
...,...,...
481,Schrodinger Inc,SDGR
482,SciPlay Corp,SCPL
483,Sea Ltd,SE
484,Seagate Technology,STX


## Get Personal Transactions

In [13]:
personal_transactions: pd.DataFrame = instance.get_personal_transactions()

personal_transactions

,name,symbol,shares
date,,,
2022-07-28,Tesla Inc.,TSLA,0.0688
2022-07-01,Apple Inc.,AAPL,0.4317
2022-08-05,3D Systems Corporation,DDD,8.8925
2022-08-05,Sibanye Stillwater Ltd,SBSW,1.1265
2022-08-18,Sibanye Stillwater Ltd,SBSW,6.5243
2022-08-18,3D Systems Corporation,DDD,5.2957
2022-08-22,SPDR S&P 500 ETF,SPY,0.0829
2022-08-22,3D Systems Corporation,DDD,2.3170
2022-08-29,Sibanye Stillwater Ltd,SBSW,-7.6507


## Get Accounts & Holdings

In [14]:
accounts: list = instance.__get_accounts__()
usd_account: Account = instance.__get_account_by_name__(account_name='EasyEquities USD')
usd_account_holdings: list = instance.__get_account_holdings__(account_name='EasyEquities USD')
usd_account_valuation: dict = instance.__get_account_valuation__(account_name='EasyEquities USD')

Exception: Login failed

In [99]:
from datetime import datetime
import requests
import json
from joblib import Parallel, delayed

def get_symbol_growth_rate(symbol: str) -> float:
    base_url: str = f'https://finance.yahoo.com/quote/{symbol}/analysis'
    http_response = requests.get(base_url, headers={
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'
    })
    json_str: str = http_response.text \
            .split('root.App.main =')[1] \
            .split('(this)')[0] \
            .split(';\n}')[0] \
            .strip()
    data = json.loads(json_str)
    symbol_quote = data['context']['dispatcher']['stores']['QuoteSummaryStore']
    growth_rate: float = [t for t in symbol_quote['earningsTrend']['trend'] if t['period'] == '+5y'][0]['growth']['raw']

    return growth_rate

def get_symbol_quote(symbol: str) -> dict:
    base_url: str = f'https://finance.yahoo.com/quote/{symbol}'
    http_response = requests.get(base_url, headers={
        'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_10_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/39.0.2171.95 Safari/537.36'
    })
    json_str: str = http_response.text \
        .split('root.App.main =')[1] \
        .split('(this)')[0] \
        .split(';\n}')[0] \
        .strip()
    data = json.loads(json_str)
    symbol_quote = data['context']['dispatcher']['stores']['QuoteSummaryStore']
    response: dict = {
        'symbol': symbol,
        'eps': symbol_quote['defaultKeyStatistics']['trailingEps']['raw'],
        'long_description': symbol_quote['summaryProfile']['longBusinessSummary'],
        'company_name': symbol_quote['price']['longName'],
        'market_cap': symbol_quote['price']['marketCap']['raw'],
        'current_price': symbol_quote['financialData']['currentPrice']['raw'],
        'total_cash': symbol_quote['financialData']['totalCash']['raw'],
        'total_debt': symbol_quote['financialData']['totalDebt']['raw'],
        'total_revenue': symbol_quote['financialData']['totalRevenue']['raw'],
        'pe_ratio': symbol_quote['summaryDetail']['trailingPE']['raw'],
        'ex_dividend_date': None,
        'dividend_date': None,
        'dividend_yield': None
    }
    
    response['growth_rate'] = get_symbol_growth_rate(symbol=symbol)

    if symbol_quote['calendarEvents']['exDividendDate'] is not None and 'raw' in symbol_quote['calendarEvents']['exDividendDate']:
        response['ex_dividend_date'] = datetime.fromtimestamp(symbol_quote['calendarEvents']['exDividendDate']['raw'])
        response['dividend_date'] = datetime.fromtimestamp(symbol_quote['calendarEvents']['dividendDate']['raw'])
        response['dividend_yield'] = symbol_quote['summaryDetail']['dividendYield']['raw']

    return response

def get_symbol_quotes_async(symbols: list) -> list:
    parrallelizer: Parallel = Parallel(n_jobs=len(symbols), prefer='threads')
    results: list = parrallelizer([delayed(get_symbol_quote)(s) for s in symbols])

    return results

In [100]:
symbols: list = ['AAPL', 'DDD', 'TSLA']
quotes: list = get_symbol_quotes_async(symbols=symbols)

In [101]:
quotes

[{'symbol': 'AAPL',
  'eps': 6.05,
  'long_description': 'Apple Inc. designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. It also sells various related services. In addition, the company offers iPhone, a line of smartphones; Mac, a line of personal computers; iPad, a line of multi-purpose tablets; AirPods Max, an over-ear wireless headphone; and wearables, home, and accessories comprising AirPods, Apple TV, Apple Watch, Beats products, HomePod, and iPod touch. Further, it provides AppleCare support services; cloud services store services; and operates various platforms, including the App Store that allow customers to discover and download applications and digital content, such as books, music, video, games, and podcasts. Additionally, the company offers various services, such as Apple Arcade, a game subscription service; Apple Music, which offers users a curated listening experience with on-demand radio stations; Apple News+